In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications import imagenet_utils
from skimage import data, color, feature

import os
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.resnet import ResNet101
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16

Using TensorFlow backend.


In [ ]:
df_train = pd.read_excel('/content/drive/My Drive/Labels.xlsx')


In [ ]:
df_train.head()

,ImageName,Level
0,fall-01-cam0-rgb-001.png,1
1,fall-01-cam0-rgb-002.png,1
2,fall-01-cam0-rgb-003.png,1
3,fall-01-cam0-rgb-004.png,1
4,fall-01-cam0-rgb-005.png,1


In [ ]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size1 = 128
im_size2 = 128

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
i = 0 
#for folder in range(9):
for f, breed in tqdm(df_train.values):
  if type(cv2.imread('IMAGES/{}'.format(f)))==type(None):
    continue 
  else: 
        img = cv2.imread('IMAGES/{}'.format(f))
        label = one_hot_labels[i]
        resizedImage = cv2.resize(img, (im_size1, im_size2))
        print(resizedImage)
        
        img_grey_scale = color.rgb2gray(resizedImage)
        canny = feature.canny(img_grey_scale, 3)

        hog_vec, hog_vis = feature.hog(resizedImage, visualize=True)
        resizedImageVec = cv2.resize(hog_vec, (im_size1, im_size2))

        x_train.append( img_grey_scale  + resizedImageVec + canny)
        #x_train.append(resizedImageVec)
        #x_train.append(canny)
        #x_train = np.expand_dims(x_train, axis=-1) 
        y_train.append(label)
        i += 1 
np.save('x_trainImgURFplusfeature',x_train) 
np.save('y_trainImgURFplusfeature',y_train) 
print('Done')

In [ ]:
i = 0 
#for folder in range(9):
for f, breed in tqdm(df_train.values):
  if type(cv2.imread('IMAGES/{}'.format(f)))==type(None):
    continue 
  else: 
    img = cv2.imread('IMAGES/{}'.format(f)) 
    label = one_hot_labels[i] 
    x_train.append(cv2.resize(img, (im_size1, im_size2))) 
    y_train.append(label) 
    i += 1 
np.save('x_trainuURF',x_train) 
np.save('y_trainURF',y_train) 
print('Done')

In [ ]:
x_train = np.load('x_trainImgURFplusfeature.npy')
y_train = np.load('y_trainImgURFplusfeature.npy')
x_train = np.expand_dims(x_train, axis=-1) 

In [ ]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(14161, 128, 128, 1)
(14161, 4)


In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import AveragePooling2D
# #Optimized ResNet-50

In [ ]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
# x = AveragePooling2D(pool_size=(4, 4))(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='RMSprop', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD', 
              metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 113s 9ms/step - loss: 1.0919 - accuracy: 0.4871 - val_loss: 1.6770 - val_accuracy: 0.3140
Epoch 2/20
12744/12744 [==============================] - 93s 7ms/step - loss: 0.6275 - accuracy: 0.7448 - val_loss: 1.8878 - val_accuracy: 0.3458
Epoch 3/20
12744/12744 [==============================] - 93s 7ms/step - loss: 0.4008 - accuracy: 0.8438 - val_loss: 0.4513 - val_accuracy: 0.8264
Epoch 4/20
12744/12744 [==============================] - 93s 7ms/step - loss: 0.3020 - accuracy: 0.8843 - val_loss: 0.3388 - val_accuracy: 0.8694
Epoch 5/20
12744/12744 [==============================] - 93s 7ms/step - loss: 0.2552 - accuracy: 0.9026 - val_loss: 0.7789 - val_accuracy: 0.7530
Epoch 6/20
12744/12744 [==============================] - 93s 7ms/step - loss: 0.2132 - accuracy: 0.9200 - val_loss: 0.2644 - val_accuracy: 0.8991
Epoch 7/20
12744/12744 [==============================] - 93s 7ms/st

In [ ]:
base_model = ResNet101(weights = None, include_top=True, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
# x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 186s 15ms/step - loss: 1.2885 - accuracy: 0.3527 - val_loss: 1.2640 - val_accuracy: 0.3641
Epoch 2/20
12744/12744 [==============================] - 153s 12ms/step - loss: 1.2399 - accuracy: 0.3599 - val_loss: 1.2552 - val_accuracy: 0.3641
Epoch 3/20
12744/12744 [==============================] - 153s 12ms/step - loss: 1.2324 - accuracy: 0.3599 - val_loss: 1.2567 - val_accuracy: 0.3641
Epoch 4/20
12744/12744 [==============================] - 153s 12ms/step - loss: 1.2318 - accuracy: 0.3599 - val_loss: 1.2570 - val_accuracy: 0.3641
Epoch 5/20
12744/12744 [==============================] - 153s 12ms/step - loss: 1.2309 - accuracy: 0.3599 - val_loss: 1.2577 - val_accuracy: 0.3641
Epoch 6/20
12744/12744 [==============================] - 153s 12ms/step - loss: 1.2319 - accuracy: 0.3599 - val_loss: 1.2575 - val_accuracy: 0.3641
Epoch 7/20
12744/12744 [==============================] -

In [ ]:
 base_model = ResNet152(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 1)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 273s 21ms/step - loss: 3.7057 - accuracy: 0.3617 - val_loss: 6002.4655 - val_accuracy: 0.3098
Epoch 2/20
12744/12744 [==============================] - 215s 17ms/step - loss: 1.9496 - accuracy: 0.4080 - val_loss: 4950.0689 - val_accuracy: 0.3098
Epoch 3/20
12744/12744 [==============================] - 215s 17ms/step - loss: 1.7370 - accuracy: 0.4684 - val_loss: 685302.1495 - val_accuracy: 0.3641
Epoch 4/20
12744/12744 [==============================] - 215s 17ms/step - loss: 1.2475 - accuracy: 0.6472 - val_loss: 3271344.3215 - val_accuracy: 0.2639
Epoch 5/20
12744/12744 [==============================] - 215s 17ms/step - loss: 0.9974 - accuracy: 0.7388 - val_loss: 544802.7608 - val_accuracy: 0.3098
Epoch 6/20
12744/12744 [==============================] - 215s 17ms/step - loss: 0.6851 - accuracy: 0.8132 - val_loss: 257469.4571 - val_accuracy: 0.3098
Epoch 7/20
12744/12744 [======

In [ ]:
base_model = VGG19(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [ ]:
 model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 79s 6ms/step - loss: 1.9635 - accuracy: 0.3438 - val_loss: 1.2634 - val_accuracy: 0.3641
Epoch 2/20
12744/12744 [==============================] - 71s 6ms/step - loss: 1.2316 - accuracy: 0.3612 - val_loss: 1.2676 - val_accuracy: 0.3641
Epoch 3/20
12744/12744 [==============================] - 71s 6ms/step - loss: 1.2312 - accuracy: 0.3570 - val_loss: 1.2595 - val_accuracy: 0.3641
Epoch 4/20
12744/12744 [==============================] - 71s 6ms/step - loss: 1.2305 - accuracy: 0.3541 - val_loss: 1.2718 - val_accuracy: 0.3641
Epoch 5/20
12744/12744 [==============================] - 71s 6ms/step - loss: 1.2310 - accuracy: 0.3559 - val_loss: 1.2604 - val_accuracy: 0.3641
Epoch 6/20
12744/12744 [==============================] - 71s 6ms/step - loss: 1.2307 - accuracy: 0.3571 - val_loss: 1.2611 - val_accuracy: 0.3641
Epoch 7/20
12744/12744 [==============================] - 71s 6ms/ste

In [ ]:
base_model = InceptionV3(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 63, 63, 32)   288         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 63, 63, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_144 (Activation)     (None, 63, 63, 32)   0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

In [ ]:
 model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 91s 7ms/step - loss: 1.8933 - accuracy: 0.3874 - val_loss: 53200.5312 - val_accuracy: 0.2639
Epoch 2/20
12744/12744 [==============================] - 67s 5ms/step - loss: 1.4358 - accuracy: 0.4280 - val_loss: 124.2545 - val_accuracy: 0.4037
Epoch 3/20
12744/12744 [==============================] - 67s 5ms/step - loss: 1.2012 - accuracy: 0.5425 - val_loss: 2.0941 - val_accuracy: 0.4340
Epoch 4/20
12744/12744 [==============================] - 67s 5ms/step - loss: 1.1636 - accuracy: 0.5549 - val_loss: 1.0273 - val_accuracy: 0.5999
Epoch 5/20
12744/12744 [==============================] - 67s 5ms/step - loss: 0.7895 - accuracy: 0.7020 - val_loss: 2.5803 - val_accuracy: 0.5949
Epoch 6/20
12744/12744 [==============================] - 67s 5ms/step - loss: 0.5253 - accuracy: 0.8106 - val_loss: 3.4766 - val_accuracy: 0.5448
Epoch 7/20
12744/12744 [==============================] - 68s 5

In [ ]:
base_model = MobileNet(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128 

In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 52s 4ms/step - loss: 1.7271 - accuracy: 0.6026 - val_loss: 1.3472 - val_accuracy: 0.3098
Epoch 2/20
12744/12744 [==============================] - 46s 4ms/step - loss: 0.6349 - accuracy: 0.7937 - val_loss: 2.4118 - val_accuracy: 0.4608
Epoch 3/20
12744/12744 [==============================] - 46s 4ms/step - loss: 0.4725 - accuracy: 0.8461 - val_loss: 2.4875 - val_accuracy: 0.6133
Epoch 4/20
12744/12744 [==============================] - 46s 4ms/step - loss: 0.3653 - accuracy: 0.8839 - val_loss: 2.1464 - val_accuracy: 0.6281
Epoch 5/20
12744/12744 [==============================] - 46s 4ms/step - loss: 0.2642 - accuracy: 0.9150 - val_loss: 1.8509 - val_accuracy: 0.6881
Epoch 6/20
12744/12744 [==============================] - 46s 4ms/step - loss: 0.2060 - accuracy: 0.9335 - val_loss: 1.1232 - val_accuracy: 0.8892
Epoch 7/20
12744/12744 [==============================] - 46s 4ms/ste

In [ ]:
base_model = Xception(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   288         input_10[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 139s 11ms/step - loss: 1.1010 - accuracy: 0.6562 - val_loss: 1.3689 - val_accuracy: 0.3641
Epoch 2/20
12744/12744 [==============================] - 127s 10ms/step - loss: 0.3742 - accuracy: 0.8681 - val_loss: 0.3808 - val_accuracy: 0.8567
Epoch 3/20
12744/12744 [==============================] - 127s 10ms/step - loss: 0.2355 - accuracy: 0.9178 - val_loss: 59.3858 - val_accuracy: 0.7212
Epoch 4/20
12744/12744 [==============================] - 127s 10ms/step - loss: 0.1704 - accuracy: 0.9459 - val_loss: 0.2849 - val_accuracy: 0.9231
Epoch 5/20
12744/12744 [==============================] - 127s 10ms/step - loss: 0.1379 - accuracy: 0.9599 - val_loss: 0.3051 - val_accuracy: 0.9033
Epoch 6/20
12744/12744 [==============================] - 127s 10ms/step - loss: 0.1127 - accuracy: 0.9655 - val_loss: 1.6189 - val_accuracy: 0.8737
Epoch 7/20
12744/12744 [==============================] 

In [ ]:
base_model = VGG16(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.1)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='SGD', 
#               metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 128, 128, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0  

In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 62s 5ms/step - loss: 466.2331 - accuracy: 0.3473 - val_loss: 1.2609 - val_accuracy: 0.3641
Epoch 2/20
12744/12744 [==============================] - 60s 5ms/step - loss: 1.2342 - accuracy: 0.3526 - val_loss: 1.2567 - val_accuracy: 0.3641
Epoch 3/20
12744/12744 [==============================] - 59s 5ms/step - loss: 1.2314 - accuracy: 0.3565 - val_loss: 1.2586 - val_accuracy: 0.3641
Epoch 4/20
12744/12744 [==============================] - 59s 5ms/step - loss: 1.2310 - accuracy: 0.3588 - val_loss: 1.2613 - val_accuracy: 0.3098
Epoch 5/20
12744/12744 [==============================] - 59s 5ms/step - loss: 1.2314 - accuracy: 0.3556 - val_loss: 1.2584 - val_accuracy: 0.3641
Epoch 6/20
12744/12744 [==============================] - 59s 5ms/step - loss: 1.2312 - accuracy: 0.3588 - val_loss: 1.2642 - val_accuracy: 0.3641
Epoch 7/20
12744/12744 [==============================] - 59s 5ms/s

In [ ]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
# x = AveragePooling2D(pool_size=(4, 4))(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
#x = Dense(1024, activation='relu')(x)
# x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='RMSprop', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD', 
              metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 1)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12744 samples, validate on 1417 samples
Epoch 1/20
12744/12744 [==============================] - 105s 8ms/step - loss: 12.2213 - accuracy: 0.4175 - val_loss: 4.9994 - val_accuracy: 0.3098
Epoch 2/20
12744/12744 [==============================] - 92s 7ms/step - loss: 2.5084 - accuracy: 0.6344 - val_loss: 37.9535 - val_accuracy: 0.3105
Epoch 3/20
12744/12744 [==============================] - 92s 7ms/step - loss: 1.8920 - accuracy: 0.7568 - val_loss: 3.9676 - val_accuracy: 0.6930
Epoch 4/20
12744/12744 [==============================] - 92s 7ms/step - loss: 1.9536 - accuracy: 0.7622 - val_loss: 3.0141 - val_accuracy: 0.6055
Epoch 5/20
12744/12744 [==============================] - 92s 7ms/step - loss: 1.1137 - accuracy: 0.8338 - val_loss: 0.9483 - val_accuracy: 0.8215
Epoch 6/20
12744/12744 [==============================] - 92s 7ms/step - loss: 2.0878 - accuracy: 0.7860 - val_loss: 1460703.6429 - val_accuracy: 0.2738
Epoch 7/20
12744/12744 [==============================] - 92